# An Introduction to Algorithms in Qiskit

In [3]:
!pip install qiskit
!pip install qiskit_algorithms
!pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 2.6 MB/s eta 0:00:00


## How is the algorithm library structured?

Qiskit offers various algorithms categorized by their tasks. For example:

- Minimum Eigensolvers: Find the smallest eigenvalue of an operator, such as the ground state energy of a chemistry Hamiltonian or an optimization problem solution expressed as an Ising Hamiltonian.
- Time Evolvers: Handle the time evolution of quantum systems.
- Amplitude Estimators: Estimate values for applications like finance.
You can find the full list of categories in the Algorithms documentation.

These algorithms are customizable and use smaller building blocks. For instance, the Variational Quantum Eigensolver (VQE) needs a trial wavefunction (a QuantumCircuit) and a classical optimizer.

Here's how to create a VQE instance:

1. TwoLocal: A parameterized circuit used as the trial wavefunction.
2. SLSQP: A classical optimizer.

You create these components separately and pass them to VQE. To try different components, just create the ones you want and use them with VQE.

In [5]:
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

num_qubits = 2
ansatz = TwoLocal(num_qubits, 'ry', 'cz')
optimizer = SLSQP(maxiter=1000)

Let's draw the ansatz so we can see it's a QuantumCircuit where θ[0] through θ[7] will be the parameters that are varied as VQE optimizer finds the minimum eigenvalue. We'll come back to the parameters later in a working example below.

In [6]:
ansatz.decompose().draw()

┌──────────┐   ┌──────────┐   ┌──────────┐   ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─■─┤ Ry(θ[2]) ├─■─┤ Ry(θ[4]) ├─■─┤ Ry(θ[6]) ├
     ├──────────┤ │ ├──────────┤ │ ├──────────┤ │ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─■─┤ Ry(θ[3]) ├─■─┤ Ry(θ[5]) ├─■─┤ Ry(θ[7]) ├
     └──────────┘   └──────────┘   └──────────┘   └──────────┘

## How to run an algorithm?

Algorithms use primitives to evaluate expectation values or sample circuits. These primitives can run on either a simulator or a real device and are interchangeable because they all follow the same interface.

For example, in the Variational Quantum Eigensolver (VQE), we need to evaluate expectation values. We can use the qiskit.primitives.Estimator, which comes with the default Qiskit Terra installation.

In [8]:
from qiskit.primitives import Estimator

estimator = Estimator()

This estimator uses an exact statevector simulation to evaluate expectation values. Alternatively, we can use shot-based and noisy simulators or real backends. For more details on simulators, check out Qiskit Aer, and for actual hardware, see Qiskit IBM Runtime.

With everything ready, we can now set up the VQE:

In [10]:
from qiskit_algorithms.minimum_eigensolvers import VQE

vqe = VQE(estimator, ansatz, optimizer)

Now we can call the compute_minimum_eigenvalue() method. This method is used by application modules like Nature and Optimization so they can work with any algorithm in the same category.

## A working example

Let's put everything together and create a complete working example. VQE will find the minimum eigenvalue (minimum energy value) of a Hamiltonian operator, so we need such an operator. The operator below was created by the Nature application module for an H2 molecule at a 0.735A interatomic distance. It's a sum of Pauli terms, but we won't go into detail about it now. The goal is to run the algorithm.

In [11]:
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list([
    ("II", -1.052373245772859),
    ("IZ", 0.39793742484318045),
    ("ZI", -0.39793742484318045),
    ("ZZ", -0.01128010425623538),
    ("XX", 0.18093119978423156)
])

In [12]:
result = vqe.compute_minimum_eigenvalue(H2_op)
print(result)

{   'aux_operators_evaluated': None,
    'cost_function_evals': 111,
    'eigenvalue': -1.857274884145482,
    'optimal_circuit': <qiskit.circuit.library.n_local.two_local.TwoLocal object at 0x7a6645f4a1d0>,
    'optimal_parameters': {   ParameterVectorElement(θ[0]): 6.661513891936551,
                              ParameterVectorElement(θ[1]): -4.8823446684827845,
                              ParameterVectorElement(θ[2]): -0.5718475845103274,
                              ParameterVectorElement(θ[3]): 3.681737007981664,
                              ParameterVectorElement(θ[4]): 4.569439129393979,
                              ParameterVectorElement(θ[5]): -1.9397076294009967,
                              ParameterVectorElement(θ[6]): 0.934445751434625,
                              ParameterVectorElement(θ[7]): 3.410555640472742},
    'optimal_point': array([ 6.66151389, -4.88234467, -0.57184758,  3.68173701,  4.56943913,
       -1.93970763,  0.93444575,  3.41055564]),
    'optimal

From the result, we can see how many cost function (energy) evaluations the optimizer needed to find the minimum eigenvalue, which is the electronic ground state energy of the H2 molecule. We can also see the optimal parameters of the ansatz used at the minimum value (approx. -1.85727).

## Updating the primitive inside VQE

Now, let's change the estimator primitive in VQE. If you're happy with the simulation results, you might want to use a shot-based simulator or run on actual hardware!

In this example, we'll switch to a shot-based estimator using Qiskit Terra's reference primitive. You could also use Qiskit Aer's estimator (qiskit_aer.primitives.Estimator) or a real backend (qiskit_ibm_runtime.Estimator).

For noisy environments, the SPSA optimizer often performs well, so we'll update the optimizer too. For more details on shot-based and noisy simulations, check out the noisy VQE tutorial.

In [14]:
from qiskit_algorithms.optimizers import SPSA

estimator = Estimator(options={"shots": 1000})

vqe.estimator = estimator
vqe.optimizer = SPSA(maxiter=100)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(result)

{   'aux_operators_evaluated': None,
    'cost_function_evals': 200,
    'eigenvalue': -1.8576551527276008,
    'optimal_circuit': <qiskit.circuit.library.n_local.two_local.TwoLocal object at 0x7a6645f48ac0>,
    'optimal_parameters': {   ParameterVectorElement(θ[0]): -0.726382967222955,
                              ParameterVectorElement(θ[1]): 7.031595933216298,
                              ParameterVectorElement(θ[2]): 1.0879438151403265,
                              ParameterVectorElement(θ[3]): 2.3731145684592834,
                              ParameterVectorElement(θ[4]): -2.5654009064749603,
                              ParameterVectorElement(θ[5]): 4.248187762835965,
                              ParameterVectorElement(θ[6]): 6.496838812930074,
                              ParameterVectorElement(θ[7]): -5.5074007013941735},
    'optimal_point': array([-0.72638297,  7.03159593,  1.08794382,  2.37311457, -2.56540091,
        4.24818776,  6.49683881, -5.5074007 ]),
    'optim

Note: We do not fix the random seed in the simulators here, so re-running gives slightly varying results.
